<a href="https://colab.research.google.com/github/LukegCashman/Cashman_ECGR4105/blob/main/LC_HW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# SECTION 0: Initialize Data Sets

import torch
import torch.optim as optim
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
from collections import OrderedDict

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [2]:
# Pull the csv file from github for Housing set
house_url = 'https://raw.githubusercontent.com/HamedTabkhi/Intro-to-ML/main/Dataset/Housing.csv'

# Read the data
house_df = pd.read_csv(house_url)

# Convert strings to numbers
house_df['mainroad'] = house_df['mainroad'].map({'yes':1 ,'no':0})
house_df['guestroom'] = house_df['guestroom'].map({'yes':1 ,'no':0})
house_df['basement'] = house_df['basement'].map({'yes':1 ,'no':0})
house_df['hotwaterheating'] = house_df['hotwaterheating'].map({'yes':1 ,'no':0})
house_df['airconditioning'] = house_df['airconditioning'].map({'yes':1 ,'no':0})
house_df['prefarea'] = house_df['prefarea'].map({'yes':1 ,'no':0})
house_df['furnishingstatus'] = house_df['furnishingstatus'].map({'furnished':1 ,'semi-furnished':0.5,'unfurnished':0})

# Print Housing to confirm there are no issues
house_df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,1.0
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,1.0
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,0.5
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,1.0
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,1.0


In [3]:
# randomize the training and validation data split for Housing set
np.random.seed(0)
housing_train, housing_validate = train_test_split(house_df, train_size = 0.8, test_size = 0.2, random_state = 100)

In [4]:
# Initialize output for Housing set
YH_train = torch.tensor(housing_train.values[:,0])
YH_valid = torch.tensor(housing_validate.values[:,0])

# Initialize input
XH_train = np.delete(housing_train, 0, axis=1)
XH_valid = np.delete(housing_validate, 0, axis=1)

# Standardize input
object = StandardScaler()
SH_train = object.fit_transform(XH_train)
SH_validate = object.fit_transform(XH_valid)

# Create ones column
HT_ones = torch.ones((436,1))
HV_ones = torch.ones((109,1))

# Combine input and ones and convert to torch
XHT_torch = torch.cat((torch.tensor(SH_train),HT_ones), -1)
XHV_torch = torch.cat((torch.tensor(SH_validate),HV_ones), -1)

# Convert to torch double
XHT = XHT_torch.float()
XHV = XHV_torch.float()
YHT = YH_train.float()
YHV = YH_valid.float()

In [5]:
# Import Cancer data set
from sklearn.datasets import load_breast_cancer

# Read the data from Cancer set
cancer = load_breast_cancer()
cancer_data = cancer.data
cancer_data.shape
cancer_input = pd.DataFrame(cancer_data)
cancer_input.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [6]:
cancer_labels = cancer.target
labels = np.reshape(cancer_labels,(569,1))
final_cancer_data = np.concatenate([cancer_input,labels],axis=1)

In [7]:
# training and validation data split for Cancer set
cancer_train, cancer_validate = train_test_split(final_cancer_data, train_size = 0.8, test_size = 0.2, random_state = 100)

In [8]:
# Initialize output for Cancer set
YC_train = torch.tensor(cancer_train[:,30])
YC_valid = torch.tensor(cancer_validate[:,30])

# Initialize input
XC_train = np.delete(cancer_train, 30, axis=1)
XC_valid = np.delete(cancer_validate, 30, axis=1)

# Standardize input
SC_train = object.fit_transform(XC_train)
SC_validate = object.fit_transform(XC_valid)

# Create ones column
CT_ones = torch.ones((455,1))
CV_ones = torch.ones((114,1))

# Combine input and ones and convert to torch
XCT_torch = torch.cat((torch.tensor(SC_train),CT_ones), -1)
XCV_torch = torch.cat((torch.tensor(SC_validate),CV_ones), -1)

# Convert to torch double
XCT = XCT_torch.float()
XCV = XCV_torch.float()
YCT = YC_train.float()
YCV = YC_valid.float()

In [9]:
# Define gradient decent function
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val, t_c_train, t_c_val, div):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train) # <1>
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val) # <1>
        loss_val = loss_fn(t_p_val, t_c_val)

        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % div == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

In [10]:
# SECTION 1: Problem 1.a

# Intialize model sequence
seq_modelH1 = nn.Sequential(OrderedDict([('hidden_linear', nn.Linear(13, 8)), ('hidden_activation', nn.Sigmoid()), ('output_linear', nn.Linear(8, 1))]))
seq_modelH1

Sequential(
  (hidden_linear): Linear(in_features=13, out_features=8, bias=True)
  (hidden_activation): Sigmoid()
  (output_linear): Linear(in_features=8, out_features=1, bias=True)
)

In [11]:
for name, param in seq_modelH1.named_parameters():
    print(name, param.shape)

hidden_linear.weight torch.Size([8, 13])
hidden_linear.bias torch.Size([8])
output_linear.weight torch.Size([1, 8])
output_linear.bias torch.Size([1])


In [12]:
# Initialize optimizer, and epochs for housing set
optimizer = optim.Adam(seq_modelH1.parameters(), lr=0.01)
n_epochs = 200;

# Run training loop
training_loop(n_epochs, optimizer, seq_modelH1, nn.MSELoss(), XHT, XHV, YHT, YHV, 10)

print()

# Adam, loss = 25158572048384.0000, (lr = 0.01) - Hidden = Sigmoid
# SGD,  loss = 3550503239680.0000, (lr = 0.01) - Hidden = Sigmoid

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([436])) that is different to the input size (torch.Size([436, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([109])) that is different to the input size (torch.Size([109, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1, Training loss 26477600964608.0000, Validation loss 25158762889216.0000
Epoch 10, Training loss 26477594673152.0000, Validation loss 25158758694912.0000
Epoch 20, Training loss 26477590478848.0000, Validation loss 25158754500608.0000
Epoch 30, Training loss 26477579993088.0000, Validation loss 25158746112000.0000
Epoch 40, Training loss 26477577895936.0000, Validation loss 25158739820544.0000
Epoch 50, Training loss 26477567410176.0000, Validation loss 25158735626240.0000
Epoch 60, Training loss 26477556924416.0000, Validation loss 25158723043328.0000
Epoch 70, Training loss 26477548535808.0000, Validation loss 25158714654720.0000
Epoch 80, Training loss 26477538050048.0000, Validation loss 25158699974656.0000
Epoch 90, Training loss 26477525467136.0000, Validation loss 25158693683200.0000
Epoch 100, Training loss 26477514981376.0000, Validation loss 25158685294592.0000
Epoch 110, Training loss 26477506592768.0000, Validation loss 25158670614528.0000
Epoch 120, Training loss 26

In [13]:
# SECTION 2: Problem 1.b

# Intialize model sequence
seq_modelH2 = nn.Sequential(
            nn.Linear(13, 256),
            nn.Tanh(),
            nn.Linear(256, 128),
            nn.Sigmoid(),
            nn.Linear(128, 64),
            nn.Tanh(),
            nn.Linear(64, 1))
seq_modelH2

Sequential(
  (0): Linear(in_features=13, out_features=256, bias=True)
  (1): Tanh()
  (2): Linear(in_features=256, out_features=128, bias=True)
  (3): Sigmoid()
  (4): Linear(in_features=128, out_features=64, bias=True)
  (5): Tanh()
  (6): Linear(in_features=64, out_features=1, bias=True)
)

In [14]:
for name, param in seq_modelH2.named_parameters():
    print(name, param.shape)

0.weight torch.Size([256, 13])
0.bias torch.Size([256])
2.weight torch.Size([128, 256])
2.bias torch.Size([128])
4.weight torch.Size([64, 128])
4.bias torch.Size([64])
6.weight torch.Size([1, 64])
6.bias torch.Size([1])


In [15]:
# Initialize optimizer, and epochs for housing set
optimizer = optim.SGD(seq_modelH2.parameters(), lr=0.000001)
n_epochs = 27000;

# Run training loop
training_loop(n_epochs, optimizer, seq_modelH2, nn.MSELoss(), XHT, XHV, YHT, YHV, 500)

print()
# min valdiation loss: 3529483747328.0000 - SGD - (256, 128, 64) (tanh, sigmoid, tanh) 27,000 epoch, higher epoch causes overfitting: Time = 177 seconds
# min validation loss: 3529486368768.0000 - SGD - (512, 256, 128) (tanh, sigmoid, tanh) 13,5000 epoch, higher epoch causes overfitting

Epoch 1, Training loss 26477598867456.0000, Validation loss 25158762889216.0000
Epoch 500, Training loss 23684293066752.0000, Validation loss 22452650377216.0000
Epoch 1000, Training loss 21220976230400.0000, Validation loss 20071372029952.0000
Epoch 1500, Training loss 19057965268992.0000, Validation loss 17985238466560.0000
Epoch 2000, Training loss 17158653345792.0000, Validation loss 16157970006016.0000
Epoch 2500, Training loss 15490889023488.0000, Validation loss 14557709860864.0000
Epoch 3000, Training loss 14026442539008.0000, Validation loss 13156521869312.0000
Epoch 3500, Training loss 12740533944320.0000, Validation loss 11929888227328.0000
Epoch 4000, Training loss 11611388510208.0000, Validation loss 10856291106816.0000
Epoch 4500, Training loss 10619901181952.0000, Validation loss 9916851945472.0000
Epoch 5000, Training loss 9749293694976.0000, Validation loss 9095018971136.0000
Epoch 5500, Training loss 8984818876416.0000, Validation loss 8376248434688.0000
Epoch 6000, T

In [16]:
# SECTION 3: Problem 2.a

# Intialize model sequence
seq_modelC1 = nn.Sequential(OrderedDict([('hidden_linear', nn.Linear(31, 8)), ('hidden_activation', nn.Sigmoid()), ('output_linear', nn.Linear(8, 1))]))
seq_modelC1

Sequential(
  (hidden_linear): Linear(in_features=31, out_features=8, bias=True)
  (hidden_activation): Sigmoid()
  (output_linear): Linear(in_features=8, out_features=1, bias=True)
)

In [17]:
for name, param in seq_modelC1.named_parameters():
    print(name, param.shape)

hidden_linear.weight torch.Size([8, 31])
hidden_linear.bias torch.Size([8])
output_linear.weight torch.Size([1, 8])
output_linear.bias torch.Size([1])


In [18]:
# Initialize optimizer, and epochs for cancer set
optimizer = optim.Adam(seq_modelC1.parameters(), lr=0.01)
n_epochs = 250;

# Run training loop
training_loop(n_epochs, optimizer, seq_modelC1, nn.MSELoss(), XCT, XCV, YCT, YCV, 10)

print()

# min valdiation loss: 0.2502 - Adam - 210 epoch, higher epoch yields same result
# min validation loss: 0.2503 - SGD - 2760 epoch, higher epoch yields same result

Epoch 1, Training loss 0.8308, Validation loss 0.7416
Epoch 10, Training loss 0.3044, Validation loss 0.2854
Epoch 20, Training loss 0.2412, Validation loss 0.2767
Epoch 30, Training loss 0.2470, Validation loss 0.2862
Epoch 40, Training loss 0.2304, Validation loss 0.2503
Epoch 50, Training loss 0.2324, Validation loss 0.2462
Epoch 60, Training loss 0.2302, Validation loss 0.2498


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([455])) that is different to the input size (torch.Size([455, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([114])) that is different to the input size (torch.Size([114, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 70, Training loss 0.2304, Validation loss 0.2532
Epoch 80, Training loss 0.2300, Validation loss 0.2507
Epoch 90, Training loss 0.2301, Validation loss 0.2496
Epoch 100, Training loss 0.2300, Validation loss 0.2505
Epoch 110, Training loss 0.2300, Validation loss 0.2507
Epoch 120, Training loss 0.2300, Validation loss 0.2503
Epoch 130, Training loss 0.2300, Validation loss 0.2503
Epoch 140, Training loss 0.2300, Validation loss 0.2505
Epoch 150, Training loss 0.2299, Validation loss 0.2504
Epoch 160, Training loss 0.2299, Validation loss 0.2503
Epoch 170, Training loss 0.2299, Validation loss 0.2504
Epoch 180, Training loss 0.2299, Validation loss 0.2504
Epoch 190, Training loss 0.2299, Validation loss 0.2504
Epoch 200, Training loss 0.2299, Validation loss 0.2504
Epoch 210, Training loss 0.2299, Validation loss 0.2503
Epoch 220, Training loss 0.2299, Validation loss 0.2503
Epoch 230, Training loss 0.2299, Validation loss 0.2503
Epoch 240, Training loss 0.2299, Validation loss 0.

In [19]:
# SECTION 4: Problem 2.b

# Intialize model sequence
seq_modelC2 = nn.Sequential(
            nn.Linear(31, 256),
            nn.Tanh(),
            nn.Linear(256, 128),
            nn.Sigmoid(),
            nn.Linear(128, 64),
            nn.Tanh(),
            nn.Linear(64, 1))

seq_modelC2

Sequential(
  (0): Linear(in_features=31, out_features=256, bias=True)
  (1): Tanh()
  (2): Linear(in_features=256, out_features=128, bias=True)
  (3): Sigmoid()
  (4): Linear(in_features=128, out_features=64, bias=True)
  (5): Tanh()
  (6): Linear(in_features=64, out_features=1, bias=True)
)

In [20]:
for name, param in seq_modelC2.named_parameters():
    print(name, param.shape)

0.weight torch.Size([256, 31])
0.bias torch.Size([256])
2.weight torch.Size([128, 256])
2.bias torch.Size([128])
4.weight torch.Size([64, 128])
4.bias torch.Size([64])
6.weight torch.Size([1, 64])
6.bias torch.Size([1])


In [21]:
# Initialize optimizer, and epochs for cancer set
optimizer = optim.Adam(seq_modelC2.parameters(), lr=0.01)
n_epochs = 200;

# Run training loop
training_loop(n_epochs, optimizer, seq_modelC2, nn.MSELoss(), XCT, XCV, YCT, YCV, 10)

print()

# min valdiation loss: 0.2503 - Adam -(256, 128, 64) (tanh, sigmoid, tanh) 1 epoch, higher epoch yields same result
# min validation loss: 0.2503 - SGD -(256, 128, 64) (tanh, sigmoid, tanh) 1 epoch, higher epoch yields same result

Epoch 1, Training loss 0.8463, Validation loss 0.7535
Epoch 10, Training loss 0.2551, Validation loss 0.2967
Epoch 20, Training loss 0.2547, Validation loss 0.2525
Epoch 30, Training loss 0.2393, Validation loss 0.2727
Epoch 40, Training loss 0.2341, Validation loss 0.2479
Epoch 50, Training loss 0.2308, Validation loss 0.2505
Epoch 60, Training loss 0.2304, Validation loss 0.2535
Epoch 70, Training loss 0.2299, Validation loss 0.2501
Epoch 80, Training loss 0.2300, Validation loss 0.2492
Epoch 90, Training loss 0.2299, Validation loss 0.2499
Epoch 100, Training loss 0.2299, Validation loss 0.2504
Epoch 110, Training loss 0.2299, Validation loss 0.2503
Epoch 120, Training loss 0.2299, Validation loss 0.2502
Epoch 130, Training loss 0.2299, Validation loss 0.2501
Epoch 140, Training loss 0.2299, Validation loss 0.2501
Epoch 150, Training loss 0.2299, Validation loss 0.2501
Epoch 160, Training loss 0.2299, Validation loss 0.2501
Epoch 170, Training loss 0.2299, Validation loss 0.2501
Epo

In [65]:
# SECTION 5: Problem 3.a

# Label class names
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [66]:
# Pull test and validation sets for Cifar 10
from torchvision import datasets, transforms
data_path = '../data-unversioned/p1ch7/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True,
                           transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))]))
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True,
                           transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))]))

Files already downloaded and verified
Files already downloaded and verified


In [67]:
n_out = 10

modelI1 = nn.Sequential(nn.Linear(3072, 128,),nn.Tanh(),nn.Linear(128, n_out,), nn.LogSoftmax(dim=1))

In [68]:
def softmax(x):
    return torch.exp(x) / torch.exp(x).sum()

In [69]:
# Run gradiant descent
learning_rate = 1e-2

optimizer = optim.SGD(modelI1.parameters(), lr=learning_rate)

loss_fn = nn.NLLLoss()

n_epochs = 100

for epoch in range(n_epochs):
    for img, label in cifar10:
        out = modelI1(img.view(-1).unsqueeze(0))
        loss = loss_fn(out, torch.tensor([label]))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 2.192430
Epoch: 1, Loss: 2.046996
Epoch: 2, Loss: 2.151026
Epoch: 3, Loss: 2.223523
Epoch: 4, Loss: 2.891221
Epoch: 5, Loss: 0.545765
Epoch: 6, Loss: 3.077375
Epoch: 7, Loss: 2.239348
Epoch: 8, Loss: 3.565413
Epoch: 9, Loss: 1.248958
Epoch: 10, Loss: 1.267157
Epoch: 11, Loss: 2.537892
Epoch: 12, Loss: 3.570691
Epoch: 13, Loss: 2.940117
Epoch: 14, Loss: 2.009115
Epoch: 15, Loss: 2.539519
Epoch: 16, Loss: 1.831364
Epoch: 17, Loss: 2.030626
Epoch: 18, Loss: 0.984725
Epoch: 19, Loss: 1.047887
Epoch: 20, Loss: 4.486320
Epoch: 21, Loss: 2.942048
Epoch: 22, Loss: 3.686979
Epoch: 23, Loss: 2.827257
Epoch: 24, Loss: 5.307379
Epoch: 25, Loss: 4.120939
Epoch: 26, Loss: 3.036174
Epoch: 27, Loss: 3.567071
Epoch: 28, Loss: 3.820597
Epoch: 29, Loss: 0.929535
Epoch: 30, Loss: 4.829077
Epoch: 31, Loss: 3.298269
Epoch: 32, Loss: 3.012058
Epoch: 33, Loss: 4.141789
Epoch: 34, Loss: 2.755024
Epoch: 35, Loss: 3.295991
Epoch: 36, Loss: 2.637544
Epoch: 37, Loss: 4.240101
Epoch: 38, Loss: 2.325

In [70]:
# Calculate validation accuracy
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        outputs = modelI1(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())

print("Accuracy: %f" % (correct / total))

Accuracy: 0.381900


In [61]:
# SECTION 5: Problem 3.b

n_out = 10

modelI2 = nn.Sequential(
            nn.Linear(3072, 256),
            nn.Tanh(),
            nn.Linear(256, 128),
            nn.Sigmoid(),
            nn.Linear(128, 64),
            nn.Tanh(),
            nn.Linear(64, n_out),
            nn.LogSoftmax(dim=1))

In [62]:
def softmax(x):
    return torch.exp(x) / torch.exp(x).sum()

In [63]:
# Run gradiant descent (12:42)
learning_rate = 1e-2

optimizer = optim.SGD(modelI2.parameters(), lr=learning_rate)

loss_fn = nn.NLLLoss()

n_epochs = 100

for epoch in range(n_epochs):
    for img, label in cifar10:
        out = modelI2(img.view(-1).unsqueeze(0))
        loss = loss_fn(out, torch.tensor([label]))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 3.494844
Epoch: 1, Loss: 3.764461
Epoch: 2, Loss: 3.488659
Epoch: 3, Loss: 4.410593
Epoch: 4, Loss: 3.775832
Epoch: 5, Loss: 2.530516
Epoch: 6, Loss: 5.754113
Epoch: 7, Loss: 4.251674
Epoch: 8, Loss: 4.840237
Epoch: 9, Loss: 2.751918
Epoch: 10, Loss: 3.841600
Epoch: 11, Loss: 4.845224
Epoch: 12, Loss: 4.152912
Epoch: 13, Loss: 4.051054
Epoch: 14, Loss: 3.600601
Epoch: 15, Loss: 2.396581
Epoch: 16, Loss: 3.859694
Epoch: 17, Loss: 2.772429
Epoch: 18, Loss: 1.884305
Epoch: 19, Loss: 3.728929
Epoch: 20, Loss: 3.693655
Epoch: 21, Loss: 4.260754
Epoch: 22, Loss: 4.650137
Epoch: 23, Loss: 4.048846
Epoch: 24, Loss: 3.058883
Epoch: 25, Loss: 5.316882
Epoch: 26, Loss: 4.310179
Epoch: 27, Loss: 3.561458
Epoch: 28, Loss: 3.999294
Epoch: 29, Loss: 5.029766
Epoch: 30, Loss: 2.310786
Epoch: 31, Loss: 3.507837
Epoch: 32, Loss: 2.982166
Epoch: 33, Loss: 3.683951
Epoch: 34, Loss: 2.723602
Epoch: 35, Loss: 4.910704
Epoch: 36, Loss: 3.697016
Epoch: 37, Loss: 3.395087
Epoch: 38, Loss: 4.423

In [64]:
# Calculate validation accuracy
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64,
                                         shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        outputs = modelI2(imgs.view(imgs.shape[0], -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())

print("Accuracy: %f" % (correct / total))

Accuracy: 0.458900
